In [1]:
!pip install nltk openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.4 MB/s eta 0:00:00


In [2]:
import pandas as pd
import openai
from openai import AzureOpenAI
import re

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import *

nltk.download('punkt')
nltk.download('stopwords')

from google.colab import drive
drive.mount('/content/drive')



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Mounted at /content/drive


In [ ]:
def clean_transcript(transcript):


In [ ]:
def cluster_complaints(df):
    clusters = df.groupby('Issue')
    cluster_info = {}

    for label, group in clusters:
        # Collect all transcripts in this cluster
        transcripts = group['Customer complaint narrative'].apply(clean_transcript).tolist()

        # Extract keywords (you can use TF-IDF or any other method)
        all_words = ' '.join(transcripts).split()
        word_freq = nltk.FreqDist(all_words)
        common_words = word_freq.most_common(10)  # Get top 10 common words

        cluster_info[label] = {
            'transcripts': transcripts,
            'keywords': [word for word, freq in common_words]
        }

    return cluster_info

In [ ]:
def create_cluster_report(cluster_info):

In [ ]:
def classify_transcripts(new_df, cluster_info):

In [ ]:
def main(data_1_path, data_2_path):